In [1]:
# Ocean Heat Budget Analyses in the Antarctica continental shelf (WAOM)

# Fabio B Dias - 4 May 2023
# Description:
#     this script get the heat transport across 1500m calculate for individual experiments:
# WAOM10: WAOM10extend_shflim_S_0.25Q_Cross-1500m-Transport.ipynb
# WAOM4: WAOM4extend_shflim_S_0.25Q_Cross-1500m-Transport.py
# WAOM4-NOTIDE: WAOM4extend_shflim_S_0.25Q_notides_Cross-1500m-Transport.py
# WAOM2:



In [2]:
# read nc output from WAOM 10km run

import xarray as xr
import pandas as p
import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import matplotlib as mpl
# mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LinearSegmentedColormap   # for custom colormaps

from datetime import datetime, timedelta

from netCDF4 import Dataset
from netCDF4 import num2date, date2num
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LinearSegmentedColormap   # for custom colormaps

import gsw

import pyresample

from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')


KeyboardInterrupt



In [ ]:
tmp_files_dir = '/g/data3/hh5/tmp/access-om/fbd581/ROMS/postprocessing/cross_contour_tmp/'

In [ ]:
# print(tmp_files_dir + expt + '_Tf_heat_transp_1500m')

In [ ]:
# load ice draft to create masks
expt = 'WAOM10'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_1500m')
Tf_heat_transport_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_1500m')
heat_transport_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_1500m')
temp_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_1500m')
z_rho_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_along_1500m')
dist_along_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lon_along_1500m')
lon_along_10km = ds.variables["one"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lat_along_1500m')
lat_along_10km = ds.variables["two"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_indices_1500m')
distance_indices_10km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

In [ ]:
# load ice draft to create masks
expt = 'WAOM10'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_CalvingFront')
Tf_heat_transport_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_CalvingFront')
heat_transport_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_CalvingFront')
temp_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_CalvingFront')
z_rho_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_along_CalvingFront')
dist_along_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lon_along_CalvingFront')
lon_along_10kmCF = ds.variables["one"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lat_along_CalvingFront')
lat_along_10kmCF = ds.variables["two"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_indices_CalvingFront')
distance_indices_10kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

In [ ]:
expt = 'WAOM4'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_1500m')
Tf_heat_transport_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_1500m')
heat_transport_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_1500m')
temp_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_1500m')
z_rho_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_along_1500m')
dist_along_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lon_along_1500m')
lon_along_4km = ds.variables["one"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lat_along_1500m')
lat_along_4km = ds.variables["two"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_indices_1500m')
distance_indices_4km = ds.variables["__xarray_dataarray_variable__"]
ds.close()

In [ ]:
expt = 'WAOM4'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_CalvingFront')
Tf_heat_transport_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_CalvingFront')
heat_transport_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_CalvingFront')
temp_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_CalvingFront')
z_rho_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_along_CalvingFront')
dist_along_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lon_along_CalvingFront')
lon_along_4kmCF = ds.variables["one"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_lat_along_CalvingFront')
lat_along_4kmCF = ds.variables["two"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_dist_indices_CalvingFront')
distance_indices_4kmCF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

In [ ]:
expt = 'WAOM4_notides'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_1500m')
Tf_heat_transport_4kmNT = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_1500m')
heat_transport_4kmNT = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_1500m')
temp_4kmNT = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_1500m')
z_rho_4kmNT = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_1500m')
z_rho_4kmNT = ds.variables["__xarray_dataarray_variable__"]
ds.close()

# ds = xr.open_dataset(tmp_files_dir + expt + '_dist_along_1500m')
# dist_along_4kmNT = ds.variables["__xarray_dataarray_variable__"]
# ds.close()

# ds = xr.open_dataset(tmp_files_dir + expt + '_lon_along_1500m')
# lon_along_4kmNT = ds.variables["one"]
# ds.close()

# ds = xr.open_dataset(tmp_files_dir + expt + '_lat_along_1500m')
# lat_along_4kmNT = ds.variables["two"]
# ds.close()

# ds = xr.open_dataset(tmp_files_dir + expt + '_dist_indices_1500m')
# distance_indicesNT = ds.variables["__xarray_dataarray_variable__"]
# ds.close()

In [ ]:
expt = 'WAOM4_notides'

ds = xr.open_dataset(tmp_files_dir + expt + '_Tf_heat_transp_CalvingFront')
Tf_heat_transport_4kmNT_CF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_heat_transp_CalvingFront')
heat_transport_4kmNT_CF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_temp_CalvingFront')
temp_4kmNT_CF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_CalvingFront')
z_rho_4kmNT_CF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

ds = xr.open_dataset(tmp_files_dir + expt + '_Zrho_CalvingFront')
z_rho_4kmNT_CF = ds.variables["__xarray_dataarray_variable__"]
ds.close()

In [ ]:
print(len(heat_transport_4km[0,:]))

In [ ]:
fig_path = '/g/data3/hh5/tmp/access-om/fbd581/ROMS/postprocessing/figs/OHB_shelf/'

# define along_contour axis with same length for both resolution:
along_4km = np.arange(0,len(heat_transport_4km[0,:]),1)
along_10km = np.arange(0,len(heat_transport_4km[0,:]),2.6734930448222567)

# some plots:
fig, ax = plt.subplots(nrows=1, figsize = (10, 5))
# plotting absolute value: Tf heat transport already subtracted during the calculation
# plt.plot(along_10km,(np.sum(np.cumsum(heat_transport_10km, axis=1),axis=0)), '-r', label='WAOM10')
plt.plot(dist_along_10km,(np.sum(np.cumsum(heat_transport_10km, axis=1),axis=0)), '-m', label='WAOM10') #'-k', label='Heat transport anomaly from $T_f$')
plt.plot(dist_along_4km,(np.sum(np.cumsum(heat_transport_4km, axis=1),axis=0)), '-b', label='WAOM4')
plt.plot(dist_along_4km,(np.sum(np.cumsum(heat_transport_4kmNT, axis=1),axis=0)), '-c', label='WAOM4-NOTIDE')

ax.set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
ax.set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
                          '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
ax.set_xlim(0, dist_along_10km[-1])
ax.set_ylim(-0.5e15, 1.5e15)

ax.set_xlabel('Longitude coordinates along contour')
ax.set_ylabel('Cumulative heat transport \n across 1500m isobath (W)');
plt.grid()
plt.legend()
name_fig='WAOM10x4x4NT_Cross-1500m_cumtransport_vint_raw.png'
plt.savefig(fig_path + name_fig, dpi=300)


In [ ]:
fig_path = '/g/data3/hh5/tmp/access-om/fbd581/ROMS/postprocessing/figs/OHB_shelf/'

# define along_contour axis with same length for both resolution:
along_4km = np.arange(0,len(heat_transport_4km[0,:]),1)
along_10km = np.arange(0,len(heat_transport_4km[0,:]),2.6734930448222567)

# some plots:
fig, ax = plt.subplots(nrows=1, figsize = (10, 5))
# plotting absolute value: Tf heat transport already subtracted during the calculation
# plt.plot(along_10km,(np.sum(np.cumsum(heat_transport_10km, axis=1),axis=0)), '-r', label='WAOM10')
plt.plot(dist_along_10kmCF, (np.sum(np.cumsum(heat_transport_10kmCF, axis=1),axis=0)), '-m', label='WAOM10') #'-k', label='Heat transport anomaly from $T_f$')
plt.plot(dist_along_4kmCF,(np.sum(np.cumsum(heat_transport_4kmCF, axis=1),axis=0)), '-b', label='WAOM4')
plt.plot(dist_along_4kmCF,(np.sum(np.cumsum(heat_transport_4kmNT_CF, axis=1),axis=0)), '-c', label='WAOM4-NOTIDE')

# ax.set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
# ax.set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
#                           '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
# ax.set_xlim(0, dist_along_10km[-1])
ax.set_ylim(-0.5e15, .5e15)

ax.set_xlabel('Longitude coordinates along contour')
ax.set_ylabel('Cumulative heat transport \n across the calving front (W)');
plt.grid()
plt.legend()
name_fig='WAOM10x4x4NT_Cross-CalvingFront_cumtransport_vint_raw.png'
plt.savefig(fig_path + name_fig, dpi=300)


In [ ]:

# define along_contour axis with same length for both resolution:
# along_4km = np.arange(0,len(heat_transport_4km[0,:]),1)
# along_10km = np.arange(0,len(heat_transport_4km[0,:]),2.6734930448222567)

[dist_along_axis_10km, Zaxis_10km] = np.meshgrid(dist_along_10km, np.arange(0,31))
[dist_along_axis_4km, Zaxis_4km] = np.meshgrid(dist_along_4km, np.arange(0,31))
[dist_along_axis_4km, Zaxis_4kmNT] = np.meshgrid(dist_along_4km, np.arange(0,31))

# define min/max for pcolor:
minHT = -.25e12
maxHT = .25e12

fig, ax = plt.subplots(nrows=4, figsize = (10, 15))
# plotting absolute value: Tf heat transport already subtracted during the calculation
ht = ax[0].pcolormesh(dist_along_axis_10km, z_rho_10km, heat_transport_10km, cmap='coolwarm', vmin=minHT, vmax=maxHT)
ax[0].set_title('WAOM10')
ht = ax[1].pcolormesh(dist_along_axis_4km, z_rho_4km, heat_transport_4km, cmap='coolwarm', vmin=minHT, vmax=maxHT)
ax[1].set_title('WAOM4')
ht = ax[2].pcolormesh(dist_along_axis_4km, z_rho_4kmNT, heat_transport_4kmNT, cmap='coolwarm', vmin=minHT, vmax=maxHT)
ax[2].set_title('WAOM4-NOTIDE')
hta = ax[3].pcolormesh(dist_along_axis_4km, z_rho_4kmNT, heat_transport_4km-heat_transport_4kmNT, cmap='coolwarm', vmin=-.1e12, vmax=.1e12)
ax[3].set_title('WAOM4 minus WAOM4-NOTIDE')

# - setting all axis xticks the same with ax.flat:
# for aa in ax.flat:
#     aa.set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
#     aa.set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
#                           '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
#     aa.set_xlim(0, dist_along_10km[-1])
# - setting individual axis xticks:
ax[0].set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
ax[0].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
                          '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
ax[0].set_xlim(0, dist_along_10km[-1])

for aa in np.arange(1,4):
    ax[aa].set_xticks(dist_along_4km[distance_indices_4km.astype(int)[:]])
    ax[aa].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
                              '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
    ax[aa].set_xlim(0, dist_along_10km[-1])

ax[3].set_xlabel('Longitude coordinates along contour')

cax1 = plt.axes([0.95, 0.31, 0.01, 0.55])
cb = plt.colorbar(ht, cax=cax1, orientation='vertical')
cb.ax.set_ylabel('Heat transport (W)', fontsize=14)

cax2 = plt.axes([0.95, 0.1, 0.01, 0.18])
cb = plt.colorbar(hta, cax=cax2, orientation='vertical')
cb.ax.set_ylabel('Heat transport anomaly (W)')

name_fig='WAOM10x4x4NT_Cross-1500m_transportXdepth.png'
plt.savefig(fig_path + name_fig, dpi=300)


In [ ]:
plt.pcolormesh(heat_transport_10kmCF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
plt.colorbar()

In [ ]:
plt.pcolormesh(heat_transport_4kmCF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
plt.colorbar()

In [ ]:
plt.pcolormesh(heat_transport_4kmNT_CF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
plt.colorbar()

In [ ]:
plt.pcolormesh(dist_along_4kmCF, z_rho_4kmNT_CF, heat_transport_4kmCF - heat_transport_4kmNT_CF)#, cmap='coolwarm', vmin=minHT*0.1, vmax=maxHT*0.1)
plt.colorbar()

In [ ]:
print(dist_along_4kmCF.shape, z_rho_4kmNT_CF.shape, heat_transport_4kmCF.shape)
# plt.plot(dist_along_4kmCF)

In [ ]:

# # define along_contour axis with same length for both resolution:
# # along_4km = np.arange(0,len(heat_transport_4km[0,:]),1)
# # along_10km = np.arange(0,len(heat_transport_4km[0,:]),2.6734930448222567)

# [dist_along_axis_10kmCF, Zaxis_10kmCF] = np.meshgrid(dist_along_10kmCF, np.arange(0,31))
# [dist_along_axis_4kmCF, Zaxis_4kmCF] = np.meshgrid(dist_along_4kmCF, np.arange(0,31))
# [dist_along_axis_4kmCF, Zaxis_4kmNT_CF] = np.meshgrid(dist_along_4kmCF, np.arange(0,31))

# # define min/max for pcolor:
# minHT = -.25e12
# maxHT = .25e12

# fig, ax = plt.subplots(nrows=4, figsize = (10, 15))
# # plotting absolute value: Tf heat transport already subtracted during the calculation
# ht = ax[0].pcolormesh(dist_along_axis_10kmCF, z_rho_10kmCF, heat_transport_10kmCF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
# ax[0].set_title('WAOM10')
# ht = ax[1].pcolormesh(dist_along_axis_4kmCF, z_rho_4kmCF, heat_transport_4kmCF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
# ax[1].set_title('WAOM4')
# ht = ax[2].pcolormesh(dist_along_axis_4kmCF, z_rho_4kmNT_CF, heat_transport_4kmNT_CF, cmap='coolwarm', vmin=minHT, vmax=maxHT)
# ax[2].set_title('WAOM4-NOTIDE')
# hta = ax[3].pcolormesh(dist_along_axis_4kmCF, z_rho_4kmNT_CF, heat_transport_4kmCF-heat_transport_4kmNT_CF, cmap='coolwarm', vmin=-.1e12, vmax=.1e12)
# ax[3].set_title('WAOM4 minus WAOM4-NOTIDE')

# # - setting all axis xticks the same with ax.flat:
# # for aa in ax.flat:
# #     aa.set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
# #     aa.set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
# #                           '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
# #     aa.set_xlim(0, dist_along_10km[-1])
# # - setting individual axis xticks:
# ax[0].set_xticks(dist_along_10kmCF[distance_indices_10kmCF.astype(int)[:]])
# ax[0].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
#                           '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
# ax[0].set_xlim(0, dist_along_10kmCF[-1])

# for aa in np.arange(1,4):
#     ax[aa].set_xticks(dist_along_4kmCF[distance_indices_4kmCF.astype(int)[:]])
#     ax[aa].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
#                               '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
#     ax[aa].set_xlim(0, dist_along_10kmCF[-1])

# ax[3].set_xlabel('Longitude coordinates along contour')

# cax1 = plt.axes([0.95, 0.31, 0.01, 0.55])
# cb = plt.colorbar(ht, cax=cax1, orientation='vertical')
# cb.ax.set_ylabel('Heat transport (W)', fontsize=14)

# cax2 = plt.axes([0.95, 0.1, 0.01, 0.18])
# cb = plt.colorbar(hta, cax=cax2, orientation='vertical')
# cb.ax.set_ylabel('Heat transport anomaly (W)')

# name_fig='WAOM10x4x4NT_Cross-CalvingFront_transportXdepth.png'
# plt.savefig(fig_path + name_fig, dpi=300)


In [ ]:

# some plots:
fig, ax = plt.subplots(nrows=2, figsize = (15, 11))
# plotting absolute value: Tf heat transport already subtracted during the calculation
# WAOM10
# ax[0].plot(dist_along_10km,(np.sum(heat_transport_10km, axis=0)), '--m', label='WAOM10',linewidth=0.2)
x=np.sum(heat_transport_10km, axis=0)
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[0].plot(dist_along_10km[np.arange(0,len(heat_transport_10km[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-m', label='Runmean WAOM10',linewidth=1) 
# WAOM4
# ax[0].plot(dist_along_4km,(np.sum(heat_transport_4km, axis=0)), '--b', label='WAOM4',linewidth=0.2)
x=np.sum(heat_transport_4km, axis=0)
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[0].plot(dist_along_4km[np.arange(0,len(heat_transport_4km[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-b', label='Runmean WAOM4',linewidth=1) 
# WAOM4-NOTIDE
# ax[0].plot(dist_along_4km,(np.sum(heat_transport_4kmNT, axis=0)), '--c', label='WAOM4-NOTIDE',linewidth=0.1)
x=np.sum(heat_transport_4kmNT, axis=0)
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[0].plot(dist_along_4km[np.arange(0,len(heat_transport_4kmNT[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-c', label='Runmean WAOM4-NOTIDE',linewidth=1) 

# - setting individual axis xticks:
ax[0].set_xticks(dist_along_10km[distance_indices_10km.astype(int)[:]])
ax[0].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
                          '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
ax[0].set_xlim(0, dist_along_10km[-1])

# WAOM10
# plotting absolute value: Tf heat transport already subtracted during the calculation
# ax[1].plot(dist_along_10kmCF,(np.sum(heat_transport_10kmCF, axis=0)), '-m', label='WAOM10',linewidth=0.2)
# x=np.sum(np.divide(heat_transport_10kmCF,10), axis=0)                                       ### DIVIDING WAOM10 BY 10!!!!!!!!!!!
x=np.sum(heat_transport_10kmCF, axis=0) 
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[1].plot(dist_along_10kmCF[np.arange(0,len(heat_transport_10kmCF[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-m', label='Runmean WAOM10',linewidth=1) 
# WAOM4
# ax[1].plot(dist_along_4kmCF,(np.sum(heat_transport_4kmCF, axis=0)), '-b', label='WAOM4',linewidth=0.2)
x=np.sum(heat_transport_4kmCF, axis=0)
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[1].plot(dist_along_4kmCF[np.arange(0,len(heat_transport_4kmCF[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-b', label='Runmean WAOM4',linewidth=1) 
# WAOM4-NOTIDE
# ax[1].plot(dist_along_4kmCF,(np.sum(heat_transport_4kmNT_CF, axis=0)), '-c', label='WAOM4-NOTIDE',linewidth=0.1)
x=np.sum(heat_transport_4kmNT_CF, axis=0)
N=20 # groups of points along the contour line - use dist to translate to distance (average throughout)
ax[1].plot(dist_along_4kmCF[np.arange(0,len(heat_transport_4kmNT_CF[0,:])-N+1)],np.convolve(x, np.ones(N)/N, mode='valid')*1e-15, '-c', label='Runmean WAOM4-NOTIDE',linewidth=1) 

# - setting individual axis xticks:
ax[1].set_xticks(dist_along_10kmCF[distance_indices_10kmCF.astype(int)[:]])
ax[1].set_xticklabels(('156$^\circ$E','160$^\circ$W', '120$^\circ$W','80$^\circ$W', '40$^\circ$W', '0$^\circ$', 
                          '40$^\circ$E', '80$^\circ$E', '120$^\circ$E'))
ax[1].set_xlim(0, dist_along_10kmCF[-1])

ax[0].set_ylabel('Cumulative heat transport \n across 1500m isobath (W)');
ax[1].set_ylabel('Cumulative heat transport \n across the Calving Front (W)');
ax[0].grid()
ax[1].grid()

ax[1].legend()
name_fig='WAOM10x4x4NT_Cross-1500m_CalvingFront_cumtransport_vint_raw.png'
plt.savefig(fig_path + name_fig, dpi=300)
